Classification is a *post hoc* decision layer **on top** of a probabilistic prediction.

From this point of view it is obvious that it is essential that, before performing classification, one should have the best possible probabilities to work with. Indeed, for cost-sensitive decisions, having good probabilities is imperative.

Standard estimator routines do not necessarily provide this, often returning a ranking/score rather than a probability, and calibration should be performed.

Here we shall calibrate a base estimator using the **isotonic regression** and **Venn-ABERS** techniques, and also calculate the results of a strictly-proper scoring rule, namely the Brier score.

Some resources:

- [What are Brier score and model calibration?](https://neptune.ai/blog/brier-score-and-model-calibration) (Neptune blog)

- [How to calibrate a classifier](https://valeman.medium.com/how-to-calibrate-your-classifier-in-an-intelligent-way-a996a2faf718) (Valeriy Manokhin)

- About [model uncertainty and conformal prediction](https://blog.dataiku.com/measuring-models-uncertainty-conformal-prediction) (Dataiku blog)

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import (
    precision_recall_fscore_support,
    roc_auc_score,
    roc_curve,
    auc,
    precision_recall_curve,
    confusion_matrix,
    brier_score_loss,
)
from catboost import Pool, CatBoostClassifier
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt

from crepes import WrapClassifier
from venn_abers import VennAbersCalibrator
from var import DATA_OUT, IMAGE_OUT, FORECAST_HOURS_IN_ADVANCE

In [2]:
df = pd.read_pickle(Path(DATA_OUT, 'df_dataset.pickle'))

In [3]:
X = df[
    [
        *[col_ for col_ in df.columns if col_.startswith('ie_')],
        *[col_ for col_ in df.columns if col_.startswith('iu_')],
        'hf',
        'hf_mav_2h',
        'f_107_adj',
        'hp_30',
        'smr',
        'solar_zenith_angle',
        'newell',
        *[col_ for col_ in df.columns if col_.startswith('local_warning_')],
        *[col_ for col_ in df.columns if col_.startswith('spectral_contribution_')],
        *[col_ for col_ in df.columns if col_.startswith('azimuth_')],
        *[col_ for col_ in df.columns if col_.startswith('velocity_')],
    ]
].copy()

y = df[f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'].copy()

In [4]:
X_train, y_train = X.loc['2014':'2020-08'].copy(), y.loc['2014':'2020-08'].copy()
X_cal, y_cal = X.loc['2020-09':'2021-06'].copy(), y.loc['2020-09':'2021-06'].copy()
X_test, y_test = X.loc['2021-07':].copy(), y.loc['2021-07':].copy()

In [5]:
cat_features = [
    'ie_variation',
    'iu_variation',
    *[col_ for col_ in df.columns if col_.startswith('local_warning_')],
]

static_params = {
    "eval_metric": "F1",
    "random_seed": 42,
    "auto_class_weights": "SqrtBalanced",
    "cat_features": cat_features,
    "od_type": "Iter",
    "use_best_model": True,
    "has_time": True,
    "od_wait": 200,
}

In [6]:
model = CatBoostClassifier(
    loss_function='Logloss',
    iterations=1_000,
    **static_params,
)

model = model.fit(
    X_train,
    y_train,
    eval_set=(X_test, y_test),
    silent=True,
)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
p, r, f, _ = precision_recall_fscore_support(y_test, y_pred)

In [ ]:
print(f'{p[1].round(3)} precision | {r[1].round(3)} recall | {f[1].round(3)} F1-score')

## Evaluation of classification

In [7]:
df_eval = X_test.copy(deep=True)
df_eval['true'] = y_test
df_eval['pred'] = model.predict(X_test)
df_eval['score'] = model.predict_proba(X_test)[:,1]

In [ ]:
# px.histogram(
#     data_frame=df_eval,
#     x=['score'],
#     color='true',
#     barmode='overlay',
# )

## Calibration curve

In [8]:
def plot_reliability_diagram(prob_pred, prob_true, scores, plot_title=None) -> go.Figure:
    """Helper function to plot a reliabilitt diagram"""
    fig = px.line(
        x=prob_pred,
        y=prob_true,
        markers=True,
    )
    fig.update_traces(
        selector=dict(type="scatter"),
        hovertemplate="Mean predicted probability: %{x:.2f}<br>Fraction of positives: %{y:.2f}",
    )

    fig.add_shape(
        type="line",
        x0=0,
        y0=0,
        x1=1,
        y1=1,
        line=dict(color="red", width=2, dash="dash"),
    )

    fig.add_trace(
        go.Histogram(
            x=scores,
            yaxis="y2",
            opacity=0.3,
            showlegend=False,
            nbinsx=25,
        )
    )
    fig.update_traces(
        selector=dict(type="histogram"),
        hovertemplate="%{y:,} samples<extra></extra>",
    )

    fig.update_layout(
        xaxis_title="Mean predicted probability",
        yaxis_title="Fraction of positives",
        yaxis2=dict(title="Count of samples", overlaying="y", side="right"),
        title=plot_title,
        template="simple_white",
    )

    return fig

In [ ]:
uncal_brier = brier_score_loss(
    y_test,
    df_eval['score'],
).round(3)

In [ ]:
prob_true, prob_pred = calibration_curve(
    df_eval['true'],
    df_eval['score'],
    n_bins=10,
)

plot_reliability_diagram(
    prob_pred=prob_pred,
    prob_true=prob_true,
    scores=df_eval["score"],
    plot_title=f'Uncalibrated model (Brier score: <b>{uncal_brier}</b>)'
)

## Isotonic calibration

In [ ]:
df_eval['score_iso_cal'] = CalibratedClassifierCV(
    estimator=model,
    method='isotonic',
    cv='prefit',
).fit(
    X_cal, y_cal,
).predict_proba(
    X_test
)[:,1]

In [ ]:
iso_brier = brier_score_loss(
    y_test,
    df_eval['score_iso_cal'],
).round(3)

In [ ]:
prob_true, prob_pred = calibration_curve(
    df_eval['true'],
    df_eval['score_iso_cal'],
    n_bins=10,
)

plot_reliability_diagram(
    prob_pred=prob_pred,
    prob_true=prob_true,
    scores=df_eval["score_iso_cal"],
    plot_title=f'Calibrated model with isotonic regression (Brier score: <b>{iso_brier}</b>)'
)

## (Inductive) Venn-ABERS calibratrion

Venn-ABERS predictors perform two isotonic regressions (using the greatest convex minorant of the cumulative sum diagram), one for for class 0 leading to probabilities $p_0$, and one for class 1 leading to probabilities $p_1$.

$p_0$ and $p_1$ form an interval within which the correct probability is deemed to be located. A single-valued probability can be obtained by combining these results, for example via

$$p \equiv \frac{p_1}{1 − p_0 + p_1}$$

Note that Inductive Venn-ABERS (IVA) needs a **disjoint hold-out calibration dataset** (`X_cal`, `y_cal`) extracted from the training data

What is a [Venn predictor](https://link.springer.com/chapter/10.1007/978-3-031-06649-8_6)?

In [ ]:
iva_cal = VennAbersCalibrator(inductive=True)

score_iva_cal, p0_p1 = iva_cal.predict_proba(
    p_cal=model.predict_proba(X_cal),
    y_cal=y_cal.values,
    p_test=model.predict_proba(X_test),
    p0_p1_output=True,
)

df_eval['score_iva_cal'] = score_iva_cal[:,1]

In [ ]:
iva_brier = brier_score_loss(
    y_test,
    df_eval['score_iva_cal'],
).round(3)

In [ ]:
prob_true, prob_pred = calibration_curve(
    df_eval['true'],
    df_eval['score_iva_cal'],
    n_bins=10,
)

plot_reliability_diagram(
    prob_pred=prob_pred,
    prob_true=prob_true,
    scores=df_eval["score_iva_cal"],
    plot_title=f'Calibrated model with Venn-ABERS (Brier score: <b>{iva_brier}</b>)'
)

## Prediction interval

In [ ]:
df_eval['score_iva_cal_up'] = p0_p1[:,1]
df_eval['score_iva_cal_lw'] = p0_p1[:,0]
df_eval['width'] = df_eval['score_iva_cal_up'] - df_eval['score_iva_cal_lw']

In [ ]:
preds = df_eval.loc['2022-03'].copy().sort_values(by=['score_iva_cal']).reset_index(drop=True)

In [ ]:
fig = go.Figure()

x = preds.index.tolist() + preds.index.tolist()[::-1]
y = preds['score_iva_cal_up'].tolist() + preds['score_iva_cal_lw'].tolist()[::-1]

fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        fill='tozeroy',
        fillcolor='rgba(0,100,80,0.4)',
        line=dict(color='rgba(0,0,0,0.2)'),
        name='Prediction interval',
    )
)

fig.add_scatter(
    x=preds.index,
    y=preds['width'],
    mode='lines',
    name="Width",
    line=dict(width=1.2, color="firebrick"),
)

fig.update_layout(
    xaxis=dict(tickvals=[], ticktext=[], title=''),
    yaxis=dict(title="Estimated probability"),
    template='plotly_white',
    legend_title='',
)

fig.show()

## crepes

In [11]:
cat_crep = WrapClassifier(model).calibrate(X_cal, y_cal)

C:\Users\vince\repos\t-fors\.venv\lib\site-packages\crepes\extras.py:71: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [np.argwhere(classes == y[i])[0][0] for i in range(len(y))])


In [13]:
cat_crep.predict_proba(X_test)

array([[0.61805802, 0.38194198],
       [0.78431706, 0.21568294],
       [0.75366501, 0.24633499],
       ...,
       [0.67796107, 0.32203893],
       [0.52786998, 0.47213002],
       [0.72111253, 0.27888747]])

In [14]:
cat_crep.predict_p(X_test)

array([[0.17014874, 0.07761232],
       [0.31993913, 0.03414622],
       [0.28337744, 0.04111305],
       ...,
       [0.21109663, 0.05974203],
       [0.12557391, 0.10524269],
       [0.24871481, 0.04951634]])

In [35]:
cat_crep.predict_set(X_test, confidence=0.90)

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 1],
       [1, 0]])

In [33]:
cat_crep.evaluate(X_test, y_test, confidence=0.90)

{'error': 0.138091985428051,
 'avg_c': 1.0487249544626593,
 'one_c': 0.9512750455373407,
 'empty': 0.0,
 'time_fit': 0.0010170936584472656,
 'time_evaluate': 1.2102313041687012}

- "error" is the  fraction of prediction sets not containing the true class label
- "avg_c" is the average no. of predicted class labels
- "one_c" is the fraction of singleton prediction sets
- "empty" is the fraction of empty prediction sets
- "time_fit" is the time taken to fit the conformal classifier
- "time_evaluate" is the time taken for the evaluation 